In [2]:
# Import necessary modules
from Data_Processing.database import connect_to_database, create_database
from Data_Processing.data_loader import load_json_data
from Data_Processing.table_creation import create_table, insert_data
import os
from dotenv import load_dotenv
load_dotenv()
password = os.getenv("PASSWORD")


In [3]:
# Connect to the default database
conn = connect_to_database(host="127.0.0.1", user="root", password=password)
conn.autocommit = True
cur = conn.cursor()

In [4]:
# Create the Company9 database
create_database(cur, "Company13")

# Close the connection to the default database
cur.close()
conn.close()

# Reconnect to the newly created Company7 database
conn = connect_to_database(host="127.0.0.1", user="root", password=password, database="Company13")
cur = conn.cursor()

In [5]:
# Define and create the necessary tables
customer_table_sql = """
CREATE TABLE IF NOT EXISTS Customer (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(255),
    country VARCHAR(100),
    city VARCHAR(100)
);
"""

ecommerce_table_sql = """
CREATE TABLE IF NOT EXISTS EcommerceWebsite (
    website_id INT PRIMARY KEY AUTO_INCREMENT,
    ecommerce_website_name VARCHAR(255) UNIQUE
);
"""

product_table_sql = """
CREATE TABLE IF NOT EXISTS Product (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255),
    product_category VARCHAR(100),
    price DECIMAL(10, 2)
);
"""

order_table_sql = """
CREATE TABLE IF NOT EXISTS Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    qty INT,
    datetime TEXT,
    ecommerce_website_name VARCHAR(255),
    FOREIGN KEY (customer_id) REFERENCES Customer(customer_id),
    FOREIGN KEY (product_id) REFERENCES Product(product_id),
    FOREIGN KEY (ecommerce_website_name) REFERENCES EcommerceWebsite(ecommerce_website_name)
);
"""

payment_table_sql = """
CREATE TABLE IF NOT EXISTS Payment (
    payment_txn_id VARCHAR(255) PRIMARY KEY,
    order_id INT,
    payment_type VARCHAR(50),
    payment_txn_success CHAR(1),
    failure_reason VARCHAR(255),
    FOREIGN KEY (order_id) REFERENCES Orders(order_id)
);
"""

In [6]:
# Execute the table creation SQL commands
create_table(cur, customer_table_sql)
create_table(cur, ecommerce_table_sql)
create_table(cur, product_table_sql)
create_table(cur, order_table_sql)
create_table(cur, payment_table_sql)

# Commit the changes
conn.commit()

In [7]:
# Load data from JSON files
customer_data = load_json_data("DataSets/customers.json")
transaction_data = load_json_data(r"DataSets/transaction_logs.json")

In [8]:
# Insert data into Customer table
customer_insert_query = """
INSERT INTO Customer (customer_id, customer_name, country, city)
VALUES (%s, %s, %s, %s);
"""
for i, row in customer_data.iterrows():
    cur.execute(customer_insert_query, list(row))
conn.commit()

In [9]:
# Prepare and insert data into Product table
product_table = transaction_data[["product_id", "product_name", "product_category", "price"]].drop_duplicates(subset=['product_id'])
product_insert_query = """
INSERT INTO Product (product_id, product_name, product_category, price)
VALUES (%s, %s, %s, %s);
"""
for i, row in product_table.iterrows():
    cur.execute(product_insert_query, list(row))
conn.commit()

In [10]:
# Prepare and insert data into EcommerceWebsite table
ecommerce_website_table = transaction_data[["ecommerce_website_name"]].drop_duplicates()
ecommerce_insert_query = """
INSERT INTO EcommerceWebsite (ecommerce_website_name)
VALUES (%s);
"""
for i, row in ecommerce_website_table.iterrows():
    cur.execute(ecommerce_insert_query, list(row))
conn.commit()


In [11]:
# Prepare and insert data into Orders table
orders_table = transaction_data[["order_id", "customer_id", "product_id", "qty", "datetime", "ecommerce_website_name"]]
order_insert_query = """
INSERT INTO Orders (order_id, customer_id, product_id, qty, datetime, ecommerce_website_name)
VALUES (%s, %s, %s, %s, %s, %s);
"""
for i, row in orders_table.iterrows():
    cur.execute(order_insert_query, list(row))
conn.commit()

In [12]:
# Prepare and insert data into Payment table
payment_table = transaction_data[["payment_txn_id", "order_id", "payment_type", "payment_txn_success", "failure_reason"]]
payment_insert_query = """
INSERT INTO Payment (payment_txn_id, order_id, payment_type, payment_txn_success, failure_reason)
VALUES (%s, %s, %s, %s, %s);
"""
for i, row in payment_table.iterrows():
    cur.execute(payment_insert_query, list(row))
conn.commit()

In [13]:

# Close the cursor and connection
cur.close()
conn.close()

In [14]:
!python Data_Processing/test_fetch_records.py Company12


Fetching records from Customer table:
   customer_id  ...               city
0          102  ...         Davidhaven
1          106  ...          Paulmouth
2          109  ...   Port Vickiemouth
3          135  ...  Port Raymondmouth
4          144  ...         Lake Jason

[5 rows x 4 columns]


Fetching records from EcommerceWebsite table:
   website_id ecommerce_website_name
0           9        www.alibaba.com
1           6         www.amazon.com
2           4          www.amazon.in
3           1        www.bestbuy.com
4          10            www.ebay.in


Fetching records from Product table:
   product_id      product_name product_category     price
0         201          Song toy             Toys  36403.00
1         202   Enjoy stationer       Stationery  26248.00
2         203     Represent toy             Toys  47723.00
3         206    Law electronic      Electronics   4177.00
4         208  Issue electronic      Electronics  25652.00


Fetching records from Orders table:
   or